# 771948_A23_T3A - Group Work Assignment #
# Part 1 - Numerical and categorical feature classification problem #
## Assignment by Chris Mintz 202369825 and Antonia Agunbiade [STUDENT ID HERE] ##

# Task 1 - Load and preprocess the dataset for the classification problem (handle missing data, convert categorical features to numerical features) #

In [11]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

df = pd.read_excel('dataset1.xlsx')

df.head(20)

,target,var1,var2,var3,var4,var5,var6,var7
0,0,509.18,417.681,Micronesia,138.0,393.00,no,2019-07-20 13:21:37
1,0,446.06,666.182,Dominica,81.0,352.05,yes,2019-04-04 21:30:46
2,1,235.50,398.097,Isle of Man,90.0,339.00,no,2019-03-03 02:59:37
3,0,306.02,518.163,Turkmenistan,102.0,439.25,yes,2019-03-19 08:00:58
4,0,453.08,600.156,Cameroon,105.0,422.95,no,2019-03-18 13:22:35
5,1,211.72,506.716,Liechtenstein,111.0,310.60,no,2019-03-18 13:00:12
6,0,401.42,627.294,French Guiana,78.0,390.05,no,2019-03-28 02:29:19
7,0,498.90,525.207,Barbados,129.0,408.75,yes,2019-06-07 05:41:16
8,1,257.90,651.209,Netherlands,147.0,280.20,no,2019-02-07 08:02:31
9,1,283.04,467.801,Chad,69.0,272.35,yes,2019-03-26 19:37:46


In [12]:
# have a look at the data types
df.info()

# quick look at the data
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925 entries, 0 to 924
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   target  925 non-null    int64  
 1   var1    925 non-null    float64
 2   var2    925 non-null    float64
 3   var3    925 non-null    object 
 4   var4    325 non-null    float64
 5   var5    925 non-null    float64
 6   var6    925 non-null    object 
 7   var7    925 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 57.9+ KB


,target,var1,var2,var4,var5
count,925.000000,925.000000,925.000000,325.000000,925.000000
mean,0.496216,360.116562,548.390134,108.452308,325.393946
std,0.500256,87.866662,135.221460,26.325744,78.862779
min,0.000000,209.560000,139.965000,57.000000,163.000000
25%,0.000000,278.640000,467.373000,87.000000,257.900000
50%,0.000000,367.640000,569.841000,105.000000,342.350000
75%,1.000000,437.600000,652.278000,126.000000,393.000000
max,1.000000,539.920000,794.848000,180.000000,457.150000


### <font color='yellow'>About this dataset</font> ###
#### var1, var2, var3, var4, var5, var6, var7 columns are features ####
#### var1, var2, var4, var5 are numerical values ####
#### var3, var6 columns are categorical values #### 
#### var 7 is a datetime ####
#### target column is the label ####

### We have null values in the data so lets enumerate them ####

In [13]:
# convert the date to proper datetime. Using coerce because there are errors in the dates
df.isnull().sum()

target      0
var1        0
var2        0
var3        0
var4      600
var5        0
var6        0
var7        0
dtype: int64

In [14]:
# some investigation shows 5 cells with an illegal datetime in them. Specifically, 5 cells have a date of 2019-02-29 which is not a valid date
def fix_not_leap_year(bad_date):
    if '2019-02-29' in str(bad_date):
        bad_date = str(bad_date).replace('2019-02-29', '2019-02-28')
        print(bad_date)
        return bad_date
    else:
        return bad_date

df['var7'] = df['var7'].apply(fix_not_leap_year)


2019-02-28 12:31:57
2019-02-28 18:06:21
2019-02-28 11:00:06
2019-02-28 23:56:06
2019-02-28 19:26:35


In [15]:
# because the the dataset is not a linear regression problem, we'll convert the datetime to features with a custom transformer
# this will allow us to extract the day, month, year and time as separate features.
# define a custom transformer function to extract datetime features
def extract_datetime_features(dates):
    dates = pd.to_datetime(dates, infer_datetime_format=True, errors='coerce')
    return pd.DataFrame({
        'year': dates.dt.year,
        'month': dates.dt.month,
        'day': dates.dt.day,
        'dayofweek': dates.dt.dayofweek,
        'hour': dates.dt.hour
    })

In [16]:
# test the datetime transformer function
date_df = extract_datetime_features(df['var7'])
print(date_df)

     year  month  day  dayofweek  hour
0    2019      7   20          5    13
1    2019      4    4          3    21
2    2019      3    3          6     2
3    2019      3   19          1     8
4    2019      3   18          0    13
..    ...    ...  ...        ...   ...
920  2019      1   22          1    12
921  2019      2    1          4    14
922  2019      6   13          3    18
923  2019      5   27          0     6
924  2019      6   18          1    17

[925 rows x 5 columns]


C:\Users\skyma\AppData\Local\Temp\ipykernel_10692\1019460975.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dates = pd.to_datetime(dates, infer_datetime_format=True, errors='coerce')


In [17]:
# DOCS When trying to deal with the datetime, we decided to convert the datetime to numerical features before putting it into the transform/fit process.
# This is because this is not a linear regression problem but a classification problem and we want to extract the day, month, year and time as separate features.
df_combined = pd.concat([df, date_df], axis=1)
print(df_combined)

     target    var1     var2          var3   var4    var5 var6  \
0         0  509.18  417.681    Micronesia  138.0  393.00   no   
1         0  446.06  666.182      Dominica   81.0  352.05  yes   
2         1  235.50  398.097   Isle of Man   90.0  339.00   no   
3         0  306.02  518.163  Turkmenistan  102.0  439.25  yes   
4         0  453.08  600.156      Cameroon  105.0  422.95   no   
..      ...     ...      ...           ...    ...     ...  ...   
920       0  422.34  547.259       Belarus    NaN  350.45  yes   
921       1  342.62  473.919         Japan    NaN  200.85   no   
922       1  265.10  538.170  Saint Martin    NaN  208.35  yes   
923       0  397.12  622.386          Chad    NaN  433.45  yes   
924       1  242.10  420.429       Albania    NaN  369.20  yes   

                    var7  year  month  day  dayofweek  hour  
0    2019-07-20 13:21:37  2019      7   20          5    13  
1    2019-04-04 21:30:46  2019      4    4          3    21  
2    2019-03-03 02:59

In [18]:
# There are too many NA in var4 to simply remove those rows so we will try to impute the missing values #
# We will have to watch the weighting of var4 feature as it will be heavily weighted to the interpolated values #

# TO DO: I don't like the interpolation of the var 4 feature. Hoping to use an algorithm that supports null data or find a regression pattern to better fit OR it's possible this is not used as feature data at all.

# Data normalization #
numerical_features = ['var1', 'var2', 'var4', 'var5', 'year', 'month', 'day', 'dayofweek', 'hour']
categorical_features = ['var3', 'var6']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])

# the result of this will be a Compressed Sparse Row (CSR) matrix which works with XGBoost and scikit-learn.
df_transformed = preprocessor.fit_transform(df_combined)

# Task 2 - Build a classifier for the classification problem using one of the specified models (logistic regression, decision trees, random forests, or artificial neural networks) #

In [22]:
# going to use XGBoost to train a decision tree model
# we will use the transformed data from the preprocessor as the input data
# target variable is the 'target' column from original df dataset
import xgboost as xgb
from sklearn.model_selection import train_test_split

# setup our features and labels
X = df_transformed
y = df['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# We're going to use the DMatrix data structure from XGBoost. This is an optimized data structure that works with XGBoost and is optimized for memory and speed
# Create the regression matrices in DMatrix format
dtrain_reg = xgb.DMatrix(X_train, y_train)
dtest_reg = xgb.DMatrix(X_test, y_test)


# Now set parameters for XGBoost
params = {
    'objective': 'reg:squarederror',  
    'max_depth': 6,
    'eta': 0.3,
    'eval_metric': 'logloss',
    'tree_method': 'hist'
}

# We'll look at the ideal number of rounds in hyperparameter tuning but for now let's use 100 rounds
n = 100
model = xgb.train(params, dtrain_reg, n)

# Task 3 - Fine tune the selected model using appropriate techniques like hyperparameter tuning, cross-validation, etc.

In [28]:
# let's look at the model's performance and see if any hyperparameter tuning is needed
from sklearn.metrics import root_mean_squared_error

predictions = model.predict(dtest_reg)
# compare the predictions to the actual values
rmse = root_mean_squared_error(y_test, predictions)
print(f'RMSE of the base model: {rmse: .3f}')

RMSE of the base model:  0.260


# Task 4 - Visualise the dataset and the model's results, where applicable like feature importance, confusion matrix, etc

# Task 5 - Report the final performance of the selected model using the appropriate performance metrics like accuracy, F1-score, etc